In [18]:
num_elements = 20000

In [19]:
import pandas as pd
from faker import Faker
fake = Faker()

In [20]:
import numpy as np
elements = np.array(["John", "James", "Alex", "Oliver", "Matt", "Tom", "Luke"])
probabilites = np.array([0.4,0.2,0.1,0.1,0.1,0.05,0.05])
male_first_names = np.random.choice(elements, int(num_elements*0.8), list(probabilites))
males = male_first_names.copy()
males[:] = "Male"

elements = np.array(["Emily", "Emma", "Sophie", "Hannah", "Kate", "Rebecca", "Chloe"])
female_first_names = np.random.choice(elements, int(num_elements*0.2), list(probabilites))
females = female_first_names.copy()
females[:] = "Female"

first_names = np.concatenate([male_first_names, female_first_names])

genders = np.concatenate([males,females])

In [21]:
elements = np.array(["Smith",
"Jones",
"Williams",
"Brown",
"Taylor",
"Davies",
"Wilson",
"Evans",
"Thomas",
"Johnson",
"Roberts",
"Walker",
"Wright",
"Thompson",
"Robinson",
"White",
"Hughes",
"Edwards",
"Hall",
"Green",
"Martin",
"Wood"])
surnames = np.random.choice(elements, num_elements)

In [22]:
elements = np.array(["London",
"Birmingham",
"Leeds",
"Glasgow",
"Sheffield",
"Bradford",
"Liverpool",
"Edinburgh",
"Manchester",
"Bristol",
"Kirklees",
"Fife"])
cities = np.random.choice(elements, num_elements)

In [23]:
target_dataset = pd.DataFrame([genders, first_names, surnames, cities]).T
target_dataset.columns = ["gender", "first_name", "surname", "city"]

In [24]:
for r in target_dataset.iterrows():
    index = r[0]
    target_dataset.loc[index,"dob"] =  fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%Y-%b-%d")
    target_dataset.loc[index,"code"] = fake.md5()

In [25]:
target_dataset.head()

,gender,first_name,surname,city,dob,code
0,Male,Tom,Thomas,Bradford,2008-Dec-18,08732eb469b61042b23d4f5e6c004b29
1,Male,Luke,Johnson,Edinburgh,1994-Nov-16,326236e04ea9affa9dd08a2897f976b8
2,Male,John,Brown,Bradford,2001-Jan-03,fa752a3ed20f2c4db032dce7321fc1bc
3,Male,Oliver,Wood,Manchester,1992-Nov-25,9a061350d043bf9828ed32ce6a76ea03
4,Male,Luke,Smith,Bristol,2003-Jan-30,e9567a0846028bd6a34c751507a3cde8


In [26]:
target_dataset = target_dataset.join(target_dataset.dob.str.split("-",expand=True))
target_dataset = target_dataset.rename(columns={0:"year", 1:"month", 2:"day"})
target_dataset = target_dataset.drop("dob", axis=1)
target_dataset.head()

,gender,first_name,surname,city,code,year,month,day
0,Male,Tom,Thomas,Bradford,08732eb469b61042b23d4f5e6c004b29,2008,Dec,18
1,Male,Luke,Johnson,Edinburgh,326236e04ea9affa9dd08a2897f976b8,1994,Nov,16
2,Male,John,Brown,Bradford,fa752a3ed20f2c4db032dce7321fc1bc,2001,Jan,03
3,Male,Oliver,Wood,Manchester,9a061350d043bf9828ed32ce6a76ea03,1992,Nov,25
4,Male,Luke,Smith,Bristol,e9567a0846028bd6a34c751507a3cde8,2003,Jan,30


In [27]:
# Now we need to create a corrupted version of this for matching
candidate_dataset = target_dataset.sample(frac=0.8,replace=False).copy()

In [28]:
import random
import string


def switch(my_string):
    pos1 = random.randrange(0,len(my_string)) 
    pos2 = random.randrange(0,len(my_string)) 
    my_string = list(my_string)
    char1 = my_string[pos1]
    char2 = my_string[pos2]
    my_string[pos1] = char2
    my_string[pos2] = char1
    return "".join(my_string)

def new_letter(my_string):
    pos1 = random.randrange(0,len(my_string)) 
    letter = random.choice(string.ascii_lowercase)
    return my_string[:pos1] + letter + my_string[pos1:]

def delete_letter(my_string):
    pos1 = random.randrange(1,len(my_string)) 
    return my_string[:pos1] + my_string[pos1+1:]

def corrupt_string(my_string, num_switches=1,num_new_letters=0,num_deletes=0):
    
    for i in range(num_switches):
        my_string = switch(my_string)
        
    for i in range(num_new_letters):
        my_string = new_letter(my_string)
        
    for i in range(num_deletes):
        my_string = delete_letter(my_string)
        
    return my_string

In [29]:
for r in candidate_dataset.iterrows():
    index = r[0]
    row = r[1]
    
    if (random.random()>0.9):
        candidate_dataset.loc[index, "first_name"] = corrupt_string(row["first_name"])
    
    if (random.random()>0.9):
        candidate_dataset.loc[index, "surname"] = corrupt_string(row["surname"], num_switches=0, num_deletes=1)
        
    if (random.random()>0.9):
        candidate_dataset.loc[index, "city"] = corrupt_string(row["city"], num_switches=0, num_deletes=1)
        
    if (random.random()>0.9):
        surname = row["surname"]
        first_name = row["first_name"]
        candidate_dataset.loc[index, "first_name"] = surname
        candidate_dataset.loc[index, "surname"] = first_name
        
    if (random.random()>0.9):
        candidate_dataset.loc[index,"year"] = fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%Y")
        
    if (random.random()>0.9):
        candidate_dataset.loc[index,"day"] = fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%d")
    
    if (random.random()>0.8):
        candidate_dataset.loc[index, "gender"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "first_name"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "surname"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "year"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "month"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "day"] = None
        

        

In [30]:
target_dataset.sample(frac=0.5).to_csv("target_dataset.csv", encoding="utf-8", index=False)

In [31]:
candidate_dataset.to_csv("candidate_dataset.csv", encoding="utf-8", index=False)